In [1]:
# Built-in modules
import subprocess
from pathlib import Path

# Third-party modules
import py3Dmol
from pymol import cmd 
from ipywidgets import interact, IntSlider

# Custom modules
from dock import AutoDockVina

MOL = '../molecules'

In [ ]:
# Enrinment construction
# conda create -n docking python=3.10
# conda install -c conda-forge -c schrodinger pymol-bundle
# pip install -U numpy vina
# pip install meeko
# pip install prody
# pip install py3Dmol
# pip install ipywidgets

## Prepare receptor protein and ligand


In [ ]:
# Import experimental structure
cmd.fetch(f'{MOL}8ibt')

'8ibt'

In [3]:
# Select chain A

cmd.select('receptor', 'chain A and not resn HOH')
cmd.save(f'{MOL}/8ibt.pdb', 'receptor')
# Select chain C (ligand)
cmd.select('ligand', 'chain C')
cmd.save(f'{MOL}/ligand.mol2', 'ligand') # mol and sdf don't work
# PDB -> PDBQT
command = f'mk_prepare_receptor.py -i {MOL}/8ibt.pdb -o {MOL}/8ibt -p'
subprocess.run(command, shell=True, capture_output=False)
command = f'mk_prepare_ligand.py -i {MOL}/ligand.mol2 -o {MOL}/ligand.pdbqt'
subprocess.run(command, shell=True, capture_output=False)

@> 5486 atoms and 1 coordinate set(s) were parsed in 0.02s.



Files written:
../molecules/8ibt.pdbqt <-- static (i.e., rigid) receptor input file
Input molecules processed: 1, skipped: 1
PDBQT files written: 1
PDBQT files not written due to error: 0
Input molecules with errors: 0


[RDKit] WARNING:[17:31:33] 8ibt: Warning - no explicit hydrogens in mol2 file but needed for formal charge estimation.


CompletedProcess(args='mk_prepare_ligand.py -i ../molecules/ligand.mol2 -o ../molecules/ligand.pdbqt', returncode=0)

## Choose docking grid

In [4]:
def read_mol(file_path: str) -> str:

    with open(file_path, "r") as f:
        pdbqt = f.readlines()
    
    # Extract only the lines starting with ATOM or HETATM
    pdb_lines = [line for line in pdbqt if line.startswith(("ATOM", "HETATM", "MODEL", "ENDMDL"))]
    pdb_str = "".join(pdb_lines)

    return pdb_str

def box2pdb(center: list, size: list, spacing = 1) -> str:
    lengths = [s * spacing for s in size]
    mins = [c - l / 2 for c, l in zip(center, lengths)]
    maxs = [c + l / 2 for c, l in zip(center, lengths)]
    corners = [
        (mins[0], mins[1], mins[2]),
        (maxs[0], mins[1], mins[2]),
        (maxs[0], maxs[1], mins[2]),
        (mins[0], maxs[1], mins[2]),
        (mins[0], mins[1], maxs[2]),
        (maxs[0], mins[1], maxs[2]),
        (maxs[0], maxs[1], maxs[2]),
        (mins[0], maxs[1], maxs[2]),
    ]

    box_pdb = ''
    for i, (x, y, z) in enumerate(corners, 1):
        box_pdb += f'ATOM      {i}   Ne BOX X   {i}    {x:8.3f}{y:8.3f}{z:8.3f}  1.00 10.00          Ne\n'

    box_pdb += '''
    CONECT    1    2
    CONECT    1    4
    CONECT    1    5
    CONECT    2    3
    CONECT    2    6
    CONECT    3    4
    CONECT    3    7
    CONECT    4    8
    CONECT    5    6
    CONECT    5    8
    CONECT    6    7
    CONECT    7    8
    '''
    return box_pdb

In [5]:
# Read pdbqt files
ligand = read_mol(f'{MOL}/ligand.pdbqt')
receptor = read_mol(f'{MOL}/8ibt.pdbqt')

# Visualize complex
view = py3Dmol.view(width=500, height=500)
view.addModel(receptor, 'pdb')
view.setStyle({'cartoon': {'color':'spectrum', 'opacity': 0.9}})
view.addModel(ligand, 'pdb')
view.addStyle({'model':-1}, {"stick": {'colorscheme': 'greenCarbon'}})
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [6]:
# Create docking grid 
center = [76, -29, -59]
size = [20, 10, 12]
box = box2pdb(center, size)

# Visualize complex + grid
view = py3Dmol.view(width=500, height=500)
view.addModel(receptor, 'pdb')
view.setStyle({'cartoon': {'color':'spectrum', 'opacity': 0.7}})
view.addModel(box, 'pdb')
view.addStyle({'elem': 'Ne'}, {'stick': {}})
view.addModel(ligand, 'pdb')
view.addStyle({'model':-1}, {"stick": {'colorscheme': 'greenCarbon'}})
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [7]:
# Run docking
receptor_file = f'{MOL}/8ibt.pdbqt'
ligand_file = f'{MOL}/ligand.pdbqt'
vina = AutoDockVina(receptor_file, ligand_file)
exhaustiveness = 30
num_poses = 20
vina.dock(
    center=center, 
    box_size=size, 
    exhaustiveness=exhaustiveness, 
    n_poses=num_poses, 
    output_name=f'{MOL}/output'
    ) 

Computing Vina grid ... Docking starting...
Docking finished. Results saved to ../molecules/output.pdbqt.
done.
Performing docking (random seed: 1176797320) ... 
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------
   1       -8.782          0          0
   2       -8.681      1.543      10.14
   3       -8.505      1.029      2.009
   4       -8.175      2.521      11.05
   5       -8.114       2.11      5.204
   6       -8.057      2.146      5.814
   7        -8.05      2.108      5.354
   8        -7.99      1.761      9.759
   9       -7.988      2.478       11.1
  10       -7.914      2.394      5.912
  11       -7.887      1.878      3.832
  12        -7.79      1.895       3.42
  13       -7.761      1.819       3.59
  14       -7.684      2.134      3.

array([-8.782, -8.681, -8.505, -8.175, -8.114, -8.057, -8.05 , -7.99 ,
       -7.988])

In [9]:
# Report results
vina.report()

Calculating RMSD...
Calculating energy...


{'rmsd': array([1.96361 , 2.87804 , 0.532889, 2.09222 , 2.04953 , 0.96538 ,
        0.721018, 2.53609 , 1.21309 , 3.04924 , 2.7661  , 1.96239 ,
        2.97532 , 1.79619 , 2.98302 , 3.21229 , 2.88488 , 2.8108  ,
        2.19771 , 2.13155 ]),
 'energy': array([-8.782, -8.681, -8.505, -8.175, -8.114, -8.057, -8.05 , -7.99 ,
        -7.988])}

In [10]:
# Create docking grid 
box = box2pdb(center, size)

# Visualize complex + grid + poses
def slider(pose: int):
    print(f'Pose {pose}')
    poses = read_mol(f'{MOL}/output.pdbqt')
    pose = poses.split('ENDMDL\n')[:-1][pose - 1]
    view = py3Dmol.view(width=500, height=500)
    view.addModel(ligand, 'pdb')
    view.addStyle({'model':-1}, {"stick": {'colorscheme': 'greenCarbon'}})
    
    view.addModel(box, 'pdb')
    view.addStyle({'elem': 'Ne'}, {'stick': {}})
    view.zoomTo()
    view.addModel(receptor, 'pdb')
    view.setStyle({'model':-1}, {'cartoon': {'color':'spectrum', 'opacity': 0.9}})
    view.addModel(pose, 'pdb')
    view.addStyle({'model':-1}, {"stick": {'colorscheme': 'pinkCarbon'}})
    
    view.show()
    

interact(slider, pose=IntSlider(min=1, max=num_poses - 1, step=1))

interactive(children=(IntSlider(value=1, description='pose', max=19, min=1), Output()), _dom_classes=('widget-…

<function __main__.slider(pose: int)>